In [4]:
import sys, os, distutils.core
sys.path.insert(0, os.path.abspath('/Users/shuai/Desktop/Mining/dev/detectron2'))

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

zsh:1: command not found: nvcc
torch:  2.5 ; cuda:  2.5.1
detectron2: 0.6


In [5]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog

In [104]:
def process_single_image(image_path, predictor, metadata):
    # Read the input image
    im = cv2.imread(image_path)
    if im is None:
        raise ValueError(f"Failed to load image at {image_path}")
    
    # Run prediction
    outputs = predictor(im)
    
    # Create visualizer
    v = Visualizer(im[:, :, ::-1],
                   metadata=metadata, 
                   scale=1, 
                   instance_mode=ColorMode.IMAGE_BW
    )
    
    # Draw predictions
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    output_image = np.array(out.get_image()[:, :, ::-1], dtype=np.uint8)

    # Get predictions
    pred_boxes = outputs["instances"].pred_boxes
    pred_classes = outputs["instances"].pred_classes

    print("pred_boxes:", pred_boxes)

    blasting_points = []

    # Draw ellipses for each detection
    for box in pred_boxes.to("cpu"):
        x_min, y_min, x_max, y_max = box.numpy()

        # Bottom-center of the bounding box
        center_x = (x_min + x_max) / 2
        center_y = y_max

        # Add blasting point
        blasting_points.append((center_x, center_y))
        print("blasting_points:", blasting_points)

        # Calculate dimensions
        radius = int((x_max - x_min) / 2)
        major_axis_length = x_max - x_min

        # Define vertical point
        vertical_point = np.array([center_x, center_y - radius, 1])

        # Apply homography
        warped_center = H @ np.array([center_x, center_y, 1])
        warped_vertical = H @ vertical_point

        # Normalize coordinates
        warped_center /= warped_center[2]
        warped_vertical /= warped_vertical[2]

        # Calculate minor axis
        minor_axis_length = 2 * np.linalg.norm(warped_vertical[:2] - warped_center[:2])

        # TODO: BUG
        # Draw ellipse
        cv2.ellipse(
            output_image,
            (int(center_x), int(center_y)),
            (int(major_axis_length / 2), int(minor_axis_length / 2)),
            0, 0, 360,
            (0, 0, 255),
            thickness=2
        )

    # Load and align design map
    design_map = cv2.imread("drill_hole_map_cv2.png", cv2.IMREAD_UNCHANGED)
    if design_map is None:
        raise ValueError("Failed to load drill_hole_map.png")

    output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2BGRA)

    # Warp design map
    aligned_map = cv2.warpPerspective(
        design_map, 
        H, 
        (output_image.shape[1], output_image.shape[0]), 
        flags=cv2.INTER_LINEAR, 
        borderMode=cv2.BORDER_CONSTANT, 
        borderValue=(0, 0, 0, 0)
    )

    # Blend images
    alpha_map = aligned_map[:, :, 3] / 255.0
    for c in range(3):
        output_image[:, :, c] = (
            alpha_map * aligned_map[:, :, c] + 
            (1 - alpha_map) * output_image[:, :, c]
        ).astype(np.uint8)

    # Combine alpha channels
    output_image[:, :, 3] = np.maximum(output_image[:, :, 3], aligned_map[:, :, 3])

    # Save output
    output_filename = "output_" + os.path.basename(image_path)
    if not cv2.imwrite(output_filename, output_image):
        raise ValueError(f"Failed to save output image to {output_filename}")
    
    return blasting_points


In [13]:
import os
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo

def setup_inference():
    # Setup configuration
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    
    # Set inference parameters
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # your number of classes
    cfg.MODEL.DEVICE = "cpu"
    cfg.INPUT.MASK_FORMAT = "bitmask"
    
    # Load your trained weights
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/Users/shuai/Desktop/Mining/dev/output/model_final.pth")
    
    # Set confidence threshold for inference
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
    
    # Create and return predictor
    predictor = DefaultPredictor(cfg)
    return predictor


In [105]:
all_blasting_points = []

image_path = "demo1_frames/demo1_frame_0004.jpg"

# Load the homography matrix from npy
H = np.load("homography_matrix.npy")

# Get the metadata (modify this according to your setup)
metadata = MetadataCatalog.get("my_dataset_val2")

predictor = setup_inference()


blasting_points = process_single_image(image_path, predictor, metadata)

all_blasting_points.extend(blasting_points)

/Users/shuai/miniconda3/envs/blast/lib/python3.12/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_locat

pred_boxes: Boxes(tensor([[965.2250, 457.2144, 981.9709, 493.7719],
        [912.2878, 432.8939, 951.4757, 496.3074],
        [869.9539, 460.0344, 906.8766, 514.9352]]))
blasting_points: [(np.float32(973.59796), np.float32(493.77185))]
blasting_points: [(np.float32(973.59796), np.float32(493.77185)), (np.float32(931.8817), np.float32(496.3074))]
blasting_points: [(np.float32(973.59796), np.float32(493.77185)), (np.float32(931.8817), np.float32(496.3074)), (np.float32(888.4152), np.float32(514.93524))]


In [100]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist


def find_related_drill_holes(transformed_df, detected_points):
    # Extract drill hole coordinates
    drill_coords = transformed_df[["X_Final", "Y_Final"]].values  # Ensure correct column names
    detected_points_np = np.array(detected_points)

    # Calculate distances between all detections and transformed drill holes
    distances = cdist(np.array(detected_points_np), drill_coords)
    
    # Find the nearest drill hole for each detected point
    results = []
    for idx, point in enumerate(detected_points):
        nearest_idx = np.argmin(distances[idx])
        
        # Add to results
        results.append({
            "Detected Point": point,
            "Drill_X": transformed_df.iloc[nearest_idx]["X_Final"],
            "Drill_Y": transformed_df.iloc[nearest_idx]["Y_Final"],
            "Pattern.Name": transformed_df.iloc[nearest_idx]["Pattern.Name"],
            "Hole.id": transformed_df.iloc[nearest_idx]["Hole.id"],
        })

    return results


In [106]:
# Load the CSV file
file_path = "transformed_drill_hole_coordinates.csv"  # Replace with your actual CSV file path
df = pd.read_csv(file_path)

# Find related drill holes
match_info = find_related_drill_holes(df, all_blasting_points)

# Print results
print(f"Nearest drill hole: {match_info}")

Nearest drill hole: [{'Detected Point': (np.float32(973.59796), np.float32(493.77185)), 'Drill_X': np.int64(977), 'Drill_Y': np.int64(495), 'Pattern.Name': 'C1_352_121', 'Hole.id': np.int64(65)}, {'Detected Point': (np.float32(931.8817), np.float32(496.3074)), 'Drill_X': np.int64(933), 'Drill_Y': np.int64(506), 'Pattern.Name': 'C1_352_121', 'Hole.id': np.int64(105)}, {'Detected Point': (np.float32(888.4152), np.float32(514.93524)), 'Drill_X': np.int64(890), 'Drill_Y': np.int64(517), 'Pattern.Name': 'C1_352_121', 'Hole.id': np.int64(145)}]


In [107]:
import cv2
import pandas as pd
import numpy as np

# Step 1: Load the transformed coordinates
transformed_coords_csv_path = "transformed_drill_hole_coordinates.csv"  # Replace with your CSV file path
transformed_coords_df = pd.read_csv(transformed_coords_csv_path)

# Step 2: Load the drone footage image
drone_image_path = "demo1_frames/demo1_frame_0000.jpg"  # Replace with your drone footage image path
drone_image = cv2.imread(drone_image_path)

# Step 3: Draw each transformed point (green) onto the drone image
for _, row in transformed_coords_df.iterrows():
    x, y = int(row["X_Final"]), int(row["Y_Final"])
    # Draw a green circle for each drill hole
    cv2.circle(drone_image, (x, y), radius=5, color=(0, 255, 0), thickness=-1)  # Green filled circle

# Step 4: Draw related points (red) from `match_info`
# Assuming match_info contains related points in a similar format
# Replace this line with your actual function call or data structure
match_info = find_related_drill_holes(transformed_coords_df, all_blasting_points)  

for match in match_info:
    x, y = int(match["Drill_X"]), int(match["Drill_Y"])
    # Draw a red circle for the related point
    cv2.circle(drone_image, (x, y), radius=5, color=(0, 0, 255), thickness=-1)  # Red filled circle

# Step 5: Save and display the result
output_image_path = "drone_image_with_drill_holes0004.png"  # Output image path
cv2.imwrite(output_image_path, drone_image)

print(f"Drone image with drill holes and related points saved to {output_image_path}")

Drone image with drill holes and related points saved to drone_image_with_drill_holes0004.png
